### Imports

In [26]:
import os
from tqdm import tqdm
import random

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import gymnasium as gym
from trade_flow.environments import metatrader

from stable_baselines3 import A2C, PPO, DQN
from stable_baselines3.common.callbacks import BaseCallback

import torch

### Create Env

#### Create Synthetics Environment

In [ ]:
from typing import List, Tuple
from datetime import datetime
from gymnasium.envs.registration import register
from trade_flow.environments.metatrader import Simulator, Timeframe, FOREX_DATA_PATH

DATA_DIR = os.path.dirname(os.getcwd())
SYNTHETIC_INDICES_DATA_PATH = os.path.join(DATA_DIR, "examples/data/synthetic_indices_symbols.joblib")
SYNTHETIC_INDICES_DATA_PATH

In [28]:
def get_data(symbols: List[str] = ["EURUSD", "GBPCAD", "USDJPY"], 
             time_range: Tuple[datetime, datetime] = (datetime(2011, 1, 1), datetime(2012, 12, 31)),
             timeframe: Timeframe = Timeframe.D1, 
             filename: str = FOREX_DATA_PATH):
    
    mt_sim = metatrader.Simulator()
    mt_sim.download_data(symbols, time_range, timeframe)
    mt_sim.save_symbols(filename)

In [29]:
# Define the time range for the data download
start_date = datetime(2020, 1, 1)
end_date = datetime(2023, 12, 31)
time_range = (start_date, end_date)

# synthetic indices

# synthetic_indices_symbols = [
#     "Volatility 10 Index", 
#     "Volatility 25 Index",  
#     "Volatility 75 (1s) Index",
#     "Volatility 150 (1s) Index",
#     "Volatility 200 (1s) Index",
#     "Volatility 250 (1s) Index"]
synthetic_indices_symbols = [
    "Step Index",
    "Volatility 25 Index",  
    "Volatility 75 (1s) Index",
    "Volatility 150 (1s) Index",]
get_data(synthetic_indices_symbols, time_range, Timeframe.H4, SYNTHETIC_INDICES_DATA_PATH)

In [ ]:

register(
    id="synthetic-indices-hedge-v0",
    entry_point="trade_flow.environments.metatrader.envs:MT5Env",
    kwargs={
        "original_simulator": Simulator(balance=1000, leverage=500, symbols_filename=SYNTHETIC_INDICES_DATA_PATH, hedge=True),
        "trading_symbols": synthetic_indices_symbols,
        "window_size": 10,
        "symbol_max_orders": 3,
        "hold_threshold": 0.3,
        "close_threshold": 0.7,
        "fee": 0.1,
    },
)

register(
    id="synthetic-indices-unhedge-v0",
    entry_point="trade_flow.environments.metatrader.envs:MT5Env",
    kwargs={
        "original_simulator": Simulator(balance=1000, leverage=500, symbols_filename=SYNTHETIC_INDICES_DATA_PATH, hedge=False),
        "trading_symbols": synthetic_indices_symbols,
        "window_size": 10,
        "hold_threshold": 0.5,
        "close_threshold": 0.5,
        "fee": 0.1,
    },
)

In [ ]:
# env_name = 'forex-hedge-v0'
# env_name = 'stocks-hedge-v0'
# env_name = 'crypto-hedge-v0'
# env_name = 'mixed-hedge-v0'

# env_name = 'forex-unhedge-v0'
# env_name = 'stocks-unhedge-v0'
# env_name = 'crypto-unhedge-v0'
# env_name = 'mixed-unhedge-v0'

env_name = 'synthetic-indices-hedge-v0'
# env_name = 'synthetic-indices-unhedge-v0'

env = gym.make(env_name)

In [ ]:
env.observation_space

In [ ]:
env.action_space

### Define Functions

In [36]:
def print_stats(reward_over_episodes):
    """  Print Reward  """

    avg = np.mean(reward_over_episodes)
    min = np.min(reward_over_episodes)
    max = np.max(reward_over_episodes)

    print (f'Min. Reward          : {min:>10.3f}')
    print (f'Avg. Reward          : {avg:>10.3f}')
    print (f'Max. Reward          : {max:>10.3f}')

    return min, avg, max


# ProgressBarCallback for model.learn()
class ProgressBarCallback(BaseCallback):

    def __init__(self, check_freq: int, verbose: int = 1):
        super().__init__(verbose)
        self.check_freq = check_freq

    def _on_training_start(self) -> None:
        """
        This method is called before the first rollout starts.
        """
        self.progress_bar = tqdm(total=self.model._total_timesteps, desc="model.learn()")

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:
            self.progress_bar.update(self.check_freq)
        return True
    
    def _on_training_end(self) -> None:
        """
        This event is triggered before exiting the `learn()` method.
        """
        self.progress_bar.close()


# TRAINING + TEST
def train_test_model(model, env, seed, total_num_episodes, total_learning_timesteps=10_000):
    """ if model=None then execute 'Random actions' """

    # reproduce training and test
    print('-' * 80)
    obs = env.reset(seed=seed)
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

    vec_env = None

    if model is not None:
        print(f'model {type(model)}')
        print(f'policy {type(model.policy)}')
        # print(f'model.learn(): {total_learning_timesteps} timesteps ...')

        # custom callback for 'progress_bar'
        model.learn(total_timesteps=total_learning_timesteps, callback=ProgressBarCallback(100))
        # model.learn(total_timesteps=total_learning_timesteps, progress_bar=True)
        # ImportError: You must install tqdm and rich in order to use the progress bar callback. 
        # It is included if you install stable-baselines with the extra packages: `pip install stable-baselines3[extra]`

        vec_env = model.get_env()
        obs = vec_env.reset()
    else:
        print ("RANDOM actions")

    reward_over_episodes = []

    tbar = tqdm(range(total_num_episodes))

    for episode in tbar:
        
        if vec_env: 
            obs = vec_env.reset()
        else:
            obs, info = env.reset()

        total_reward = 0
        done = False

        while not done:
            if model is not None:
                action, _states = model.predict(obs)
                obs, reward, done, info = vec_env.step(action)
            else: # random
                action = env.action_space.sample()
                obs, reward, terminated, truncated, info = env.step(action)
                done = terminated or truncated

            total_reward += reward
            if done:
                break

        reward_over_episodes.append(total_reward)

        if episode % 10 == 0:
            avg_reward = np.mean(reward_over_episodes)
            tbar.set_description(f'Episode: {episode}, Avg. Reward: {avg_reward:.3f}')
            tbar.update()

    tbar.close()
    avg_reward = np.mean(reward_over_episodes)

    return reward_over_episodes

### Train + Test Env

In [ ]:
seed = 2024  # random seed
total_num_episodes = 50

print ("env_name                 :", env_name)
print ("seed                     :", seed)

# INIT matplotlib
plot_settings = {}
plot_data = {'x': [i for i in range(1, total_num_episodes + 1)]}


#### Random actions

In [ ]:
# Random actions
model = None 
total_learning_timesteps = 0
rewards = train_test_model(model, env, seed, total_num_episodes, total_learning_timesteps)
min, avg, max = print_stats(rewards)
class_name = f'Random actions'
label = f'Avg. {avg:>7.2f} : {class_name}'
plot_data['rnd_rewards'] = rewards
plot_settings['rnd_rewards'] = {'label': label}

#### SB3 Agents

In [ ]:
learning_timesteps_list_in_K = [25]
# learning_timesteps_list_in_K = [50, 250, 500]
# learning_timesteps_list_in_K = [500, 1000, 3000, 5000]

# RL Algorithms: https://stable-baselines3.readthedocs.io/en/master/guide/algos.html
model_class_list = [A2C, PPO, DQN]

for timesteps in learning_timesteps_list_in_K:
    total_learning_timesteps = timesteps * 1000
    step_key = f'{timesteps}K'

    for model_class in model_class_list:
        policy_dict = model_class.policy_aliases
        # https://stable-baselines3.readthedocs.io/en/master/guide/custom_policy.html
        policy = policy_dict.get('MultiInputPolicy') # Try MlpPolicy or MlpLstmPolicy

        try:
            model = model_class(policy, env, verbose=0)
            class_name = type(model).__qualname__
            plot_key = f'{class_name}_rewards_'+step_key
            rewards = train_test_model(model, env, seed, total_num_episodes, total_learning_timesteps)
            min, avg, max, = print_stats(rewards)
            label = f'Avg. {avg:>7.2f} : {class_name} - {step_key}'
            plot_data[plot_key] = rewards
            plot_settings[plot_key] = {'label': label}     

        except Exception as e:
            print(f"ERROR: {str(e)}")
            continue

### Plot Results

In [ ]:
data = pd.DataFrame(plot_data)

sns.set_style('whitegrid')
plt.figure(figsize=(8, 6))

for key in plot_data:
    if key == 'x':
        continue
    label = plot_settings[key]['label']
    line = plt.plot('x', key, data=data, linewidth=1, label=label)

plt.xlabel('episode')
plt.ylabel('reward')
plt.title('Random vs. SB3 Agents')
plt.legend()
plt.show()